In [ ]:
# Configure Dask Cluster
from dask_gateway import Gateway
gateway = Gateway(
     "http://a9e5f257a48ee11ea98031262d9f6c99-fe7a75363d0599a6.elb.us-east-1.amazonaws.com",
     proxy_address="tls://a9e5e36ac48ee11ea98031262d9f6c99-0e5bdfca23b2f6e8.elb.us-east-1.amazonaws.com:8786",
     auth="jupyterhub",
)
cluster = gateway.new_cluster()
cluster.scale(10)

In [ ]:
cluster

In [ ]:
# Get Dask Client
client = cluster.get_client()
client

In [ ]:
# Import Libraries
import os
import xarray as xr
from satsearch import Search

In [ ]:
# Find Landsat8 Data
search = Search.search(
    collection='landsat-8-l1',
    bbox=[-110, 39.5, -105, 40.5],
    property=["eo:cloud_cover<50", "eo:row=033", "eo:column=037"]
)
items = search.items()
print(items.calendar())

In [ ]:
# Build xarray object
band_2s=[]
band_3s=[]
band_4s=[]
band_5s=[]
for item in items:
    band_2 = f"s3://landsat-pds{item.assets['B2']['href'][36:]}"
    band_3 = f"s3://landsat-pds{item.assets['B3']['href'][36:]}"
    band_4 = f"s3://landsat-pds{item.assets['B4']['href'][36:]}"
    band_5 = f"s3://landsat-pds{item.assets['B5']['href'][36:]}"
    band_2ds = xr.open_rasterio(band_2, chunks={'x': 1000, 'y': 1000})
    band_3ds = xr.open_rasterio(band_3, chunks={'x': 1000, 'y': 1000})
    band_4ds = xr.open_rasterio(band_4, chunks={'x': 1000, 'y': 1000})
    band_5ds = xr.open_rasterio(band_5, chunks={'x': 1000, 'y': 1000})

    band_2ds = band_2ds.assign_coords(eoband=2)
    band_2ds = band_2ds.assign_coords(time=item['datetime'])
    band_2ds = band_2ds.expand_dims('time')
    band_2ds = band_2ds.expand_dims('eoband')

    band_3ds = band_3ds.assign_coords(eoband=3)
    band_3ds = band_3ds.assign_coords(time=item['datetime'])
    band_3ds = band_3ds.expand_dims('time')
    band_3ds = band_3ds.expand_dims('eoband')
    
    band_4ds = band_4ds.assign_coords(eoband=4)
    band_4ds = band_4ds.assign_coords(time=item['datetime'])
    band_4ds = band_4ds.expand_dims('time')
    band_4ds = band_4ds.expand_dims('eoband')
    
    band_5ds = band_5ds.assign_coords(eoband=5)
    band_5ds = band_5ds.assign_coords(time=item['datetime'])
    band_5ds = band_5ds.expand_dims('time')
    band_5ds = band_5ds.expand_dims('eoband')
    
    band_2s.append(band_2ds)
    band_3s.append(band_3ds)
    band_4s.append(band_4ds)
    band_5s.append(band_5ds)

band_2ds = xr.concat(band_2s, 'time')
band_3ds = xr.concat(band_3s, 'time')
band_4ds = xr.concat(band_4s, 'time')
band_5ds = xr.concat(band_5s, 'time')

ds = xr.concat([band_2ds, band_3ds, band_4ds, band_5ds], 'eoband')
ds.data

In [ ]:
# Plot one timestep of band 4
ds.sel(eoband=4, time='2019-05-02T18:01:51.125149+00:00', band=1).data
%time ds.sel(eoband=4, time='2019-05-02T18:01:51.125149+00:00', band=1).plot.imshow()

In [ ]:
# Plot mean of band 4
ds.sel(eoband=4, band=1).data
%time ds.sel(eoband=4, band=1).mean('time').plot.imshow()

In [ ]:
# Calculate NDVI for one timestep
time = '2019-05-02T18:01:51.125149+00:00'

band_4_ds = ds.sel(eoband=4, time=time, band=1)
band_5_ds = ds.sel(eoband=5, time=time, band=1)

In [ ]:
%time NDVI = (band_5_ds - band_4_ds) / (band_5_ds + band_4_ds)
NDVI.data

In [ ]:
%time NDVI.plot.imshow()

In [ ]:
# Calculate mean NDVI
band_4_ds = ds.sel(eoband=4, band=1)
band_5_ds = ds.sel(eoband=5, band=1)

In [ ]:
%time NDVI = (band_5_ds - band_4_ds) / (band_5_ds + band_4_ds)
NDVI.data

In [ ]:
%time NDVI.mean('time').plot.imshow()